#Задание

На вебинаре мы говорили что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для задачи сантимент анализа на данных с вебинара
1. построить свёрточные архитектуры
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN и (RNN -> CNN)
4. сдлать выводы что получилось лучше

In [28]:
import matplotlib.pyplot as plt

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
path = '/content/drive/MyDrive/Colab Notebooks/GB/Introduction to Natural Language Processing/lesson_8/'

In [2]:
!pip install stop_words

  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32912 sha256=c6383b69a410f685b7ae87c157b99e47c707bee4d32d4507786fd0ba67e06d69
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [10]:
!pip install pymorphy2
from pymorphy2 import MorphAnalyzer

     |████████████████████████████████| 55 kB 1.6 MB/s 
     |████████████████████████████████| 8.2 MB 5.1 MB/s 


In [11]:
# попробуем запрограммировать простую рекурентную сеть. Возьмем датасет с прошлого занятия

import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

df_train = pd.read_csv(f"{path}/data/train.csv")
df_test = pd.read_csv(f"{path}/data/test.csv")
df_val = pd.read_csv(f"{path}/data/val.csv")



In [12]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [13]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [15]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from tensorflow.keras.metrics import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [16]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [17]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)



In [20]:
y_train = df_train['class'].values
y_val = df_val['class'].values

In [24]:
result = {'name': list(), 'score': list()}

#SimpleRNN

In [18]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 44s 131ms/step - loss: 0.5563 - accuracy: 0.7045 - val_loss: 0.4884 - val_accuracy: 0.7569


In [22]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 19ms/step - loss: 0.5191 - accuracy: 0.7369


Test score: 0.5191256403923035
Test accuracy: 0.7368513941764832


In [25]:
result['name'].append('SimpleRNN')
result['score'].append(score[0])

#LSTM

Какие проблемы у рекурентных сетей?

- затухают градиенты
- медленно, нужно всегда дойти до конца

Как решить? -> LSTM


<img src="images/lstm.png">


https://colah.github.io/posts/2015-08-Understanding-LSTMs/


Давайте, кратко посмотрим как это работает:


<img src="images/LSTMMaths.png">

In [26]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 82s 247ms/step - loss: 0.5554 - accuracy: 0.7069 - val_loss: 0.4935 - val_accuracy: 0.7536


In [30]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 2s 37ms/step - loss: 0.5088 - accuracy: 0.7458


Test score: 0.5088440775871277
Test accuracy: 0.7458449006080627


In [31]:
result['name'].append('LSTM')
result['score'].append(score[0])

#GRU

Какие проблемы:

- вычислительно сложно -> медленнее
- на очень длинных последовательностях все равно затухает градиент


Зачем платить больше - уберем некоторые врата (точнее совместим) -> ускоримся, уменьшим число параметров -> GRU


<img src="images/gru.png">


GRU Math


<img src="images/GRUMath.png">


In [32]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 73s 220ms/step - loss: 0.5541 - accuracy: 0.7062 - val_loss: 0.4944 - val_accuracy: 0.7533


In [35]:
# plt.figure(figsize=(20,12))
# plt.plot(history.history['accuracy'], '-.', label = f'accuracy', color='m')
# plt.plot(history.history['val_accuracy'], '-', label = f'val_accuracy', color='g')
        
# plt.legend()
# plt.grid('On')
# plt.title('Accuracy of epochs')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.show()

In [36]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 32ms/step - loss: 0.5082 - accuracy: 0.7461


Test score: 0.508188784122467
Test accuracy: 0.7460653185844421


In [37]:
result['name'].append('GRU')
result['score'].append(score[0])

In [63]:
df_result = pd.DataFrame(result)

#Result

In [72]:
df_result

,name,score
0,SimpleRNN,0.519126
1,LSTM,0.508844
2,GRU,0.508189


3 подхода:

<img src="images/RNNCompar.png">


Как регуляризовать?
- дропаут
- рекурентный дропаут


<img src="images/Dropouts.png">